In [1]:
# @title Requisitos :
%pip install torch torchvision torchaudio
%pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\pedro parentoni\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\pedro parentoni\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import glob

# Caminho base do dataset
base_path = "Dataset-news-articles-pt-br-main"

# Mapeia categorias para rótulos numéricos conforme a ordem fornecida
category_labels = {
    "ciencia e tecnologia": 0,
    "economia": 1,
    "entretenimento": 2,
    "esportes": 3,
    "politica": 4
}

# Listas para armazenar textos e rótulos
texts = []
labels = []

# Percorre todas as subpastas dentro do diretório base
for category in os.listdir(base_path):
    category_path = os.path.join(base_path, category)
    
    # Verifica se é um diretório e se a categoria está mapeada
    if os.path.isdir(category_path) and category in category_labels:
        label = category_labels[category]  # Obtém o rótulo numérico da categoria
        
        # Pega todos os arquivos .txt dentro da categoria
        for file_path in glob.glob(os.path.join(category_path, "*.txt")):
            with open(file_path, "r", encoding="utf-8") as file:
                texts.append(file.read())  # Adiciona o texto
                labels.append(label)  # Adiciona o rótulo correspondente

# Exemplo de saída
print(f"Total de textos carregados: {len(texts)}")
print(f"Exemplo de texto: {texts[401][:400]}")
print(f"Rótulo correspondente: {labels[401]} ({list(category_labels.keys())[labels[401]]})")


Total de textos carregados: 500
Exemplo de texto: Autonomia do BC pode ser alterada na Câmara

Aprovado pelo Senado, o projeto de autonomia do Banco Central pode sofrer modificações na Câmara. O partido Novo se prepara para tentar enxugar a proposta que recebeu aval dos senadores para reduzir os chamados acessórios que foram colocados para o BC.

O texto mantém o controle dos preços como objetivo central, mas inclui ainda duas novas metas acessór
Rótulo correspondente: 4 (politica)


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Carregar tokenizer e modelo
model_name = "bert-base-multilingual-cased"  # Escolha o modelo adequado
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:

print("Número de textos:", len(texts))
print("Número de labels:", len(labels))


Número de textos: 500
Número de labels: 500


In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim import AdamW

# Tokenizar textos com tamanho fixo
max_length = 512  # Limita o número de tokens
inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Converter labels para tensor
labels_tensor = torch.tensor(labels, dtype=torch.long)  # Converte para tensor

# Criar dataset
dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], labels_tensor)

# Definir proporção de treino e teste
train_size = int(0.8 * len(dataset))  # 80% para treino
test_size = len(dataset) - train_size

# Dividir dataset aleatoriamente
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Criar dataloaders para treino e teste
train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=False)

# Definir otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Mover modelo para GPU se disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [7]:
# Loop de treinamento
epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # Zerar gradientes
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Época {epoch + 1}, Perda: {total_loss:.4f}")

# Avaliação no conjunto de teste
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")


Época 1, Perda: 86.8029
Época 2, Perda: 36.0070
Época 3, Perda: 33.5038
Época 4, Perda: 26.4037
Época 5, Perda: 21.9383
Época 6, Perda: 18.3819
Época 7, Perda: 31.2513
Época 8, Perda: 22.4218
Época 9, Perda: 13.5351
Época 10, Perda: 14.5338
Acurácia no conjunto de teste: 0.8900


In [16]:
noticia = "A Secretaria da Receita Federal divulgou nesta quarta-feira (12) as regras do Imposto de Renda 2025, ano-base 2024. O prazo de entrega vai de 17 de março até 30 de maio neste ano.Quem não entregar dentro do prazo fixado está sujeito a uma multa mínima de R$ 165,74 e a um valor máximo correspondente a 20`%` do imposto sobre a renda devido. "


# Tokenizar o texto corretamente
inputs_teste = tokenizer([noticia], padding=True, truncation=True, return_tensors="pt").to(device)

# Rodar no modelo
model.eval()
with torch.no_grad():
    outputs = model(**inputs_teste)

# Obter rótulos previstos
predicoes = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# Mapear rótulos para categorias
category_labels = {
    "ciencia e tecnologia": 0,
    "economia": 1,
    "entretenimento": 2,
    "esportes": 3,
    "politica": 4
}

# Inverter o dicionário para mapear índice -> nome da categoria
index_to_category = {v: k for k, v in category_labels.items()}

# Garantir que a previsão está dentro do intervalo válido
resultados = [index_to_category.get(p, "Desconhecido") for p in predicoes]

# Definir rótulos reais (exemplo; ajuste conforme os rótulos corretos)
labels = ["economia"]  # O rótulo esperado para esse texto

# Calcular número de acertos
acertos = sum(1 for p, r in zip(resultados, labels) if p == r)
total = len(labels)  # Deve ser o número de exemplos, não o número de caracteres

precisao = (acertos / total) * 100 if total > 0 else 0  # Evitar divisão por zero

# Mostrar resultados
print(f"Texto: {noticia[:100]}... → Categoria prevista: {resultados[0]}")
print(f"\nNúmero de acertos: {acertos}/{total} ({precisao:.2f}%)")

Texto: A Secretaria da Receita Federal divulgou nesta quarta-feira (12) as regras do Imposto de Renda 2025,... → Categoria prevista: economia

Número de acertos: 1/1 (100.00%)


In [15]:
torch.save(model.state_dict(), "modelo_treinado.pth")


Acima ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ - Treinamento do modelo
--------------------------------------------------------------------------------------------------------------
Abaixo vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv - Modelo Carregado

In [28]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Carregar tokenizer e modelo
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
# carregar os pesos
model.load_state_dict(torch.load("modelo_treinado.pth"))
model.to(device)
model.eval()  # Modo de avaliação


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [36]:
noticia = ["""Trump ameaça taxar em 200% os vinhos europeus se UE não retirar tarifa de 50% sobre uísque dos EUA
Bloco de países europeus decidiu retaliar EUA por conta das tarifas impostas por Trump, mas presidente não gostou e voltou a ameaçar a região com novas tarifas.
O presidente dos Estados Unidos, Donald Trump, disse nesta quinta-feira (13) que colocaria uma tarifa de 200% sobre todos os vinhos e outros produtos alcoólicos que saíssem da União Europeia se o bloco não removesse sua tarifa de 50% sobre o uísque americano.

A Comissão Europeia informou, na quarta-feira (12), que vai retaliar os EUA, aplicando contratarifas sobre 26 bilhões de euros (US$ 28 bilhões) em produtos vindos do país (como o uísque) a partir do mês que vem, intensificando uma guerra comercial global em resposta às tarifas gerais dos EUA sobre aço e alumínio.

A União Europeia garantiu, no entanto, que permanece aberta a negociações e que acredita que tarifas maiores não são do interesse de ninguém.

Trump, no entanto, não falou sobre renegociar sua política tarifária e atacou o bloco nas redes sociais.

"A União Europeia, uma das autoridades tributárias e tarifárias mais hostis e abusivas do mundo, que foi formada com o único propósito de tirar vantagem dos Estados Unidos, acaba de colocar uma tarifa desagradável de 50% sobre o uísque", escreveu Trump na plataforma Truth Social.

"Se esta Tarifa não for removida imediatamente, os EUA em breve colocarão uma Tarifa de 200% sobre todos os VINHOS, CHAMPANHE E PRODUTOS ALCOÓLICOS SAINDO DA FRANÇA E OUTROS PAÍSES REPRESENTADOS PELA UE. Isso será ótimo para os negócios de Vinho e Champanhe nos EUA."
A União Europeia exportou 4,9 bilhões de euros em vinhos para os EUA no ano passado, de acordo com a agência de estatísticas Eurostat. Isso representa 29% das exportações totais de vinho do bloco. De todas as exportações, a França representou quase metade dos negócios, e a Itália quase 40%.

O hiperfoco de Trump em tarifas abalou a confiança dos investidores, consumidores e empresas e aumentou os temores de recessão. Ele também desgastou as relações com o Canadá, um aliado próximo e grande parceiro comercial, ao ameaçar repetidamente anexar o país vizinho.

A Comissão Europeia disse que encerrará sua suspensão atual de tarifas sobre produtos dos EUA em 1º de abril e que suas novas tarifas estarão totalmente em vigor até 13 de abril.

As indústrias de bebidas e cosméticos da Europa reagiram ao plano da Comissão Europeia de aplicar tarifas sobre produtos feitos nos EUA, dizendo que isso colocava em risco uma parte importante do comércio do continente com o país norte-americano nesses setores.

Outros produtos visados ​​pela União Europeia além do uísque incluem produtos industriais e agrícolas, como aço e alumínio, têxteis, eletrodomésticos, plásticos, aves, carne bovina, ovos, laticínios, açúcar e vegetais.

Com a declaração, os índices dos mercados de ações dos EUA caíram, assim como as ações de fabricantes de bebidas alcoólicas europeias.

A Comissão Europeia disse que encerrará sua suspensão atual de tarifas sobre produtos dos EUA em 1º de abril e que suas novas tarifas estarão totalmente em vigor até 13 de abril.

Sem ceder 'de forma alguma'
Nesta quinta-feira (13), Trump também afirmou que ele não vai mudar de ideia sobre a imposição das tarifas recíprocas de importação previstas para abril.

"Nós fomos enganados por anos e não seremos enganados", afirmou ele durante uma reunião no Salão Oval com o secretário-geral da Organização do Tratado do Atlântico Norte (Otan), Mark Rutte.""",""" Imposto de Renda 2025: declaração começa em 17 de março e se estende até 30 de maio
Receita Federal divulgou as regras do IR deste ano. Programa estará disponível nesta quinta e restituições começam em 30 de maio. Fisco espera receber 46,2 milhões de declarações neste ano.
Por Alexandro Martello, Thiago Resende, g1 e TV Globo — Brasília

12/03/2025 15h12  Atualizado há 21 horas. Atualmente, cerca de 22% da população paga Imposto de Renda, e com a mudança da regra serão só 8%, destaca Luiza Nassif Pires, do Made-USP — Foto: Marcello Casal Jr./Agência Brasil

A Secretaria da Receita Federal divulgou nesta quarta-feira (12) as regras do Imposto de Renda 2025, ano-base 2024. O prazo de entrega vai de 17 de março até 30 de maio neste ano.

Quem não entregar dentro do prazo fixado está sujeito a uma multa mínima de R$ 165,74 e a um valor máximo correspondente a 20% do imposto sobre a renda devido.

Quem é obrigado a declarar o Imposto de Renda em 2024
quem recebeu rendimentos tributáveis acima de R$ 33.888,00 em 2024. O valor é um pouco maior do que o da declaração do IR do ano passado (R$ 30.639,90) por conta da ampliação da faixa de isenção;
contribuintes que receberam rendimentos isentos, não-tributáveis ou tributados exclusivamente na fonte, cuja soma tenha sido superior a R$ 200 mil no ano passado;
quem obteve, em qualquer mês de 2024, ganho de capital na alienação de bens ou direitos, sujeito à incidência do imposto, ou realizou operações em bolsas de valores, de mercadorias, de futuros e assemelhadas cuja soma foi superior a R$ 40 mil, ou com apuração de ganhos líquidos sujeitas à incidência do imposto;
quem teve isenção de imposto sobre o ganho de capital na venda de imóveis residenciais, seguido de aquisição de outro imóvel residencial no prazo de 180 dias;
quem teve, em 2024, receita bruta em valor superior a R$ 169.440,00 em atividade rural;
quem tinha, até 31 de dezembro de 2024, a posse ou a propriedade de bens ou direitos, inclusive terra nua, de valor total superior a R$ 800 mil;
quem passou para a condição de residente no Brasil em qualquer mês e se encontrava nessa condição até 31 de dezembro de 2024;
quem optou por declarar os bens, direitos e obrigações detidos pela entidade controlada, direta ou indireta, no exterior como se fossem detidos diretamente pela pessoa física;
Possui trust no exterior;
quem atualizou bens imóveis pagando ganho de capital diferenciado em dezembro/2024 (Lei nº 14.973/2024);
quem auferiu rendimentos no exterior de aplicações financeiras e de lucros e dividendos;
Deseja atualizar bens no exterior.
Restituições
A Receita prioriza a data de entrega das declarações e também observa uma fila de prioridades para alguns grupos, que recebem a restituição antes de todo o resto (mesmo que tenham entregado a declaração nos últimos dias do prazo).

Quem envia a declaração mais cedo recebe a restituição primeiro. Por outro lado, se houver erros ou omissões na entrega, o contribuinte perde a posição na fila — ou seja, vai para o fim do calendário de restituições.

Têm prioridade na restituição do Imposto de Renda, nesta ordem:

idosos acima de 80 anos;
idosos entre 60 e 79 anos;
contribuintes com alguma deficiência física ou mental ou moléstia grave;
contribuintes cuja maior fonte de renda seja o magistério;
utilizaram a pré-preenchida E optaram por receber a restituição por PIX;
contribuintes que adotarem a declaração pré-preenchida OU optarem por receber a restituição via PIX.
Para receber via PIX, é preciso que a chave informada no momento da declaração seja o CPF do contribuinte. PIX vinculados ao e-mail ou ao telefone, por exemplo, não podem ser usados.

""", """Governo define suas indicações para o conselho de administração da Petrobras
Sleição pela assembleia de acionistas está marcada para 16 de abril. Geralmente, o governo consegue eleger seis dos 11 conselheiros da Petrobras.
O governo decidiu os nomes a serem indicados para o Conselho de Administração da Petrobras, cuja eleição pela assembleia de acionistas está marcada para 16 de abril, segundo apurou o g1.

Geralmente, o governo consegue eleger seis dos 11 conselheiros da Petrobras. As outras vagas são ocupadas pelos acionistas minoritários e pelo indicado pelos trabalhadores da estatal.

🔎 O Conselho de Administração da Petrobras é responsável por definir e aprovar o plano estratégico da companhia, além de eleger a diretoria executiva e fiscalizar a gestão da companhia. É esse colegiado que aprova investimentos da estatal.

Os nomes definidos pelo governo já fazem parte do colegiado e serão indicados para recondução. Só há uma mudança: a indicação de José Fernando Coura no lugar de Vitor Saback, atual secretário de Geologia, Mineração e Transformação Mineral do Ministério de Minas e Energia.

Novo indicado para o conselho da Petrobras, Coura já foi presidente da Fiemg e do Instituto Brasileiro de Mineração (Ibram).

O g1 apurou que o executivo foi indicado no lugar de Saback porque o secretário está considerando uma proposta de emprego na diretoria de uma subsidiária. Se aceitar a proposta, Saback também deve sair do ministério.

Veja a lista dos conselheiros que devem ser reconduzidos pelo governo:

Pietro Adamo, secretário de Petróleo do Ministério de Minas e Energia: indicado para presidir o Conselho de Administração;
Magda Chambriard, presidente da Petrobras;
Bruno Moretti, secretário de Análise Governamental da Casa Civil;
Rafael Dubeux, secretário executivo adjunto do Ministério da Fazenda;
Renato Campos Galuppo, ex-assessor jurídico da Câmara dos Deputados.
Indicado para a diretoria da Agência Nacional de Petróleo, Gás Natural e Biocombustíveis (ANP), Pietro Adamo deve continuar como presidente do conselho da Petrobras até ter o seu nome aprovado pelo Senado Federal para integrar a agência reguladora."""
, """Consignado CLT: entenda como funciona empréstimo para trabalhadores do setor privado
Objetivo do programa é facilitar o crédito para quem tem carteira assinada, permitindo o uso de parte do FGTS como garantia. Saiba quem tem direito e como vai funcionar.
Por Guilherme Mazui, Alexandro Martello, Kevin Lima, g1 — Brasília

13/03/2025 12h42  Atualizado há 2 horas

O governo Lula lançou nesta quarta-feira (12) uma linha de crédito consignado para trabalhadores com carteira assinada do setor privado.

A medida beneficiará também empregados rurais, domésticos e empregados de MEI (microempreendedor individual). O objetivo é baratear o crédito e estimular a economia.

Com o novo programa, batizado de "Crédito do Trabalhador", quem tem carteira assinada poderá contratar essa modalidade de empréstimo usando até 10% do saldo do Fundo de Garantia por Tempo de Serviço (FGTS) como garantia. Também pode usar 100% da multa rescisória na demissão sem justa causa (que é de 40% do valor do saldo).

Com o FGTS como garantia do consignado, o governo prevê que a taxa de juros caia cerca de 40%, quase a metade do que é cobrado hoje. Em dezembro do ano passado, por exemplo, a taxa média de juros no consignado do setor privado foi de 2,89% ao mês. (leia mais abaixo).

🔎Do ponto de vista do governo, a medida vai baratear o crédito e estimular a economia com queda na taxa de juros. Especialistas, contudo, alertam para possibilidade de maior endividamento.

Quem a medida beneficia?
Segundo o Ministério do Trabalho, o crédito beneficiará trabalhadores formais da iniciativa privada, o que inclui empregados rurais, domésticos e empregados de MEI (microempreendedor individual).

Os trabalhadores poderão utilizar a Carteira de Trabalho Digital para buscar empréstimos em mais de 80 instituições financeiras que operam junto ao Instituto Nacional do Seguro Social (INSS).

De quanto será a redução nos juros?
Com as garantias do FGTS atreladas ao consignado, o governo prevê que a taxa de juros caia cerca de 40%, quase pela metade.

Em dezembro do ano passado, a taxa média de juros cobrada no consignado do setor privado foi de 2,89% ao mês, enquanto que a taxa média dos servidores públicos ficou em 1,8% ao mês, e, dos aposentados do INSS, em 1,66% ao mês.

Segundo o governo, o país tem atualmente 47 milhões de trabalhadores formais, o que inclui 2,2 milhões de trabalhadores domésticos, 4 milhões de trabalhadores rurais e empregados do MEI, até então excluídos dos empréstimos consignados.

Com essa mudança, o volume de crédito disponível para trabalhadores do setor privado pode triplicar, saindo dos atuais R$ 40 bilhões para R$ 120 bilhões.

MP já está valendo?
A MP tem efeito de lei assim que publicada no "Diário Oficial da União" (DOU), mas precisa ser aprovada pelo Congresso Nacional em até 120 dias para continuar valendo.

O governo ainda precisa definir os detalhes da regulamentação e finalizar o desenvolvimento da plataforma digital para operacionalizar o crédito — prevista para entrar em operação dia 21 de março.

Segundo o Ministério do Trabalho, será preciso regulamentar o uso de até 10% do saldo do FGTS e da multa em caso de demissão como garantia dos empréstimos. A expectativa é que isso seja feito até junho.

Como aderir?
Os trabalhadores poderão acessar uma plataforma para analisar as ofertas de empréstimos, comparando, por exemplo, as taxas de juros.

O sistema entrará em operação pelos bancos a partir de 21 de março e, quem já tem um consignado ativo, poderá migrar para a nova linha a partir de 25 de abril.

A portabilidade entre os bancos poderá ser realizada a partir de 6 de junho.

Com o novo programa, todos os trabalhadores com carteira assinada poderão contratar essa modalidade de empréstimo, podendo usar até 10% do saldo do Fundo de Garantia por Tempo de Serviço (FGTS) como garantia e, também, 100% da multa rescisória na demissão sem justa causa (de 40% do valor do saldo).

Quem fez uso do Saque-Aniversário do FGTS vai poder contratar?
Segundo o ministro do Trabalho, Luiz Marinho, sim. O trabalhador que utilizou o Saque-Aniversário poderá contratar o empréstimo consignado privado.

"O Saque-Aniversário não está na mesa, é outro departamento completamente diferente. Aqui é o crédito folha de pagamento. O mesmo cidadão que antecipou o Saque-Aniversário, poderá constituir empréstimo? Sim. Poderá"
E se o trabalhador mudar de emprego?
Segundo o governo, caso o trabalhador mude de emprego, o desconto em folha para quitar as parcelas do empréstimo terá de ser feito pelo novo empregador.

Como deve funcionar o novo consignado?
Os bancos terão acesso às informações dos trabalhadores do eSocial para facilitar a concessão de crédito consignado, uma vez que poderão avaliar melhor o risco das operações.

🔹 CTPS Digital: por meio do aplicativo da Carteira de Trabalho Digital, o trabalhador solicita a proposta de crédito às instituições financeiras habilitadas pelo governo. O trabalhador autoriza o acesso a dados como nome, CPF, margem do salário disponível para consignação e tempo de empresa.

🔹 Propostas: após solicitar o crédito, o trabalhador recebe ofertas dos bancos em até 24h. O trabalhador poderá comparar ofertas e escolher a opção mais vantajosa.

🔹Datas: o sistema via CTPS Digital entra em operação a partir de 21 de março. A partir de 25 de abril, o trabalhador poderá também iniciar contratações pelos canais eletrônicos dos bancos.

🔹Migração e portabilidade: quem já tem um consignado ativo pode fazer a migração para a nova linha a partir de 25 de abril. A portabilidade entre os bancos poderá ser realizada a partir de 6 de junho.

🔹 Comprometimento de até 35% do salário bruto: O limite inclui benefícios, abonos e comissões.

🔹 FGTS como garantia: O trabalhador poderá contratar essa modalidade de empréstimo, podendo usar até 10% do saldo do Fundo de Garantia por Tempo de Serviço (FGTS) como garantia e, também, 100% da multa rescisória na demissão sem justa causa (de 40% do valor do saldo).

🔹 Caso de demissão: O trabalhador poderá usar os 10% do saldo do FGTS e a multa por rescisão para quitar a dívida com o banco. Caso o valor não seja suficiente, há uma interrupção no pagamento, que será retomada quando o trabalhador conseguir outro emprego CLT, o que acarretará em correções. Também será possível procurar o banco para acertar uma nova forma de pagamento.

🔹 Desconto automático: O empregador será responsável por descontar a parcela do salário e repassá-la à Caixa Econômica Federal, que fará o pagamento aos bancos credores."""

]

# Tokenizar o texto corretamente
inputs_teste = tokenizer(noticia, padding=True, truncation=True, return_tensors="pt").to(device)

# Rodar no modelo
model.eval()
with torch.no_grad():
    outputs = model(**inputs_teste)

# Obter rótulos previstos
predicoes = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# Mapear rótulos para categorias
category_labels = {
    "ciencia e tecnologia": 0,
    "economia": 1,
    "entretenimento": 2,
    "esportes": 3,
    "politica": 4
}

# Inverter o dicionário para mapear índice -> nome da categoria
index_to_category = {v: k for k, v in category_labels.items()}

# Garantir que a previsão está dentro do intervalo válido
resultados = [index_to_category.get(p, "Desconhecido") for p in predicoes]

# Criar lista de rótulos esperados, assumindo que todas são de economia
labels = ["economia" for _ in noticia]

# Calcular número de acertos
total = len(labels)  # Agora temos um rótulo esperado por notícia
acertos = sum(1 for p, r in zip(resultados, labels) if p == r)
precisao = (acertos / total) * 100 if total > 0 else 0  # Evitar divisão por zero

# Mostrar resultados corretamente para todas as notícias
for i, (texto, categoria) in enumerate(zip(noticia, resultados)):
    print(f"Notícia {i+1}: {texto[:100]}... → Categoria prevista: {categoria}")

print(f"\nNúmero de acertos: {acertos}/{total} ({precisao:.2f}%)")


Notícia 1: Trump ameaça taxar em 200% os vinhos europeus se UE não retirar tarifa de 50% sobre uísque dos EUA
B... → Categoria prevista: economia
Notícia 2:  Imposto de Renda 2025: declaração começa em 17 de março e se estende até 30 de maio
Receita Federal... → Categoria prevista: economia
Notícia 3: Governo define suas indicações para o conselho de administração da Petrobras
Sleição pela assembleia... → Categoria prevista: politica
Notícia 4: Consignado CLT: entenda como funciona empréstimo para trabalhadores do setor privado
Objetivo do pro... → Categoria prevista: economia

Número de acertos: 3/4 (75.00%)
